# K近邻分类算法
## 公式笔记

* Lp距离公式
$$ 
L_{p}\left(x_{i}, x_{j}\right)=\left(\sum_{l=1}^{n}\left|x_{i}^{(l)}-x_{j}^{(l)}\right|^{p}\right)^{\frac{1}{p}}
$$

* $ p=2 $ 为欧氏距离
$$ 
L_{2}\left(x_{i}, x_{j}\right)=\left(\sum_{l=1}^{n}\left|x_{i}^{(l)}-x_{j}^{(l)}\right|^{2}\right)^{\frac{1}{2}}
 $$

* $ p=1 $ 为曼哈顿距离
$$ 
L_{1}\left(x_{i}, x_{j}\right)=\sum_{l=1}^{n}\left|x_{i}^{(l)}-x_{j}^{(l)}\right|
 $$

## KNNClassifier实现


In [129]:
import numpy as np
class kdTreeNode:
    def __init__(self, x, x_index, parent):
        '''
        所有节点共用x
        利用索引来表示该节点所覆盖的数据
        '''
        self.x = x
        self.x_index = x_index
        self.parent = parent
        self.edge_nodes_x_index = None
        self.dim_index = None
        self.left = None
        self.right = None
class kdTree:
    def __init__(self, distance_method = 'o'):
        self.dim_len = None
        self.x = None
        self.root = None
        if(distance_method == 'm'):
            self._dist_method = self._manhattan_distance
            self._dist_method_2d = self._manhattan_distance_2d
        else:
            self._dist_method = self._euler_dist
            self._dist_method_2d = self._euler_dist_2d
    def _euler_dist(self, x1, x):
        return np.sqrt(np.multiply(x-x1, x-x1).sum())
    def _manhattan_distance(self, x1, x):
        return np.abs(x-x1).sum()
    def _euler_dist_2d(self, x2d, x):
        return np.sum(np.multiply(x2d-x, x2d-x), axis = 1)
    def _manhattan_distance_2d(self, x2d, x):
        return np.sum(np.abs(x2d-x), axis = 1)
    def _median(self, arr):
        temp = arr.copy()
        temp.sort()
        return temp[len(temp) // 2]
    def _build_child_node(self, x, x_index, parent):
        if(0 == x_index.size):
            return None
        else:
            return kdTreeNode(x, x_index, parent)
    def _fit(self, dim_len, current_dim,  node):
        if(None == node):
            return
        node.dim_index = current_dim
        splited_x = node.x[node.x_index]
        median_value = self._median(splited_x[:, current_dim])   
        node.edge_nodes_x_index = splited_x[np.where(splited_x[:, current_dim] 
                                           == median_value)][:, -1]
        node.left = self._build_child_node(node.x, splited_x[np.where(splited_x[:, current_dim] 
                                           < median_value)][:, -1], node)
        node.right = self._build_child_node(node.x, splited_x[np.where(splited_x[:, current_dim] 
                                           > median_value)][:, -1], node)
        self._fit(dim_len, (current_dim + 1)%dim_len, node.left)
        self._fit(dim_len, (current_dim + 1)%dim_len, node.right)

    def fit(self, x):
        self.dim_len = x.shape[1]
        #保存原始索引
        self.x = np.hstack((x, np.arange(len(x)).reshape(-1, 1)))        
        self.root = kdTreeNode(self.x, np.arange(len(x)), None)
        self._fit(self.dim_len, 0, self.root)
    def _search_nreaest_leaf(self, target_x, node):
        #print('dim', node.dim_index, 'edge value', edge_value)
        #print(node.x[node.edge_nodes_x_index])
        if(node.left != None or node.right != None):
            edge_value = node.x[node.edge_nodes_x_index][0][node.dim_index]
            if(target_x[node.dim_index] == edge_value):
                if(None != node.left):
                    return self._search_nreaest_leaf(target_x, node.left)
                if(None != node.right):
                    return self._search_nreaest_leaf(target_x, node.right)
            elif(target_x[node.dim_index] < edge_value):
                if(None != node.left):
                    return self._search_nreaest_leaf(target_x, node.left)
                else:
                    return self._search_nreaest_leaf(target_x, node.right)
            else:
                if(None != node.right):
                    return self._search_nreaest_leaf(target_x, node.right)
                else:
                    return self._search_nreaest_leaf(target_x, node.left)
        else:
            return node
    def _search_nreaest_node(self, target_x, leaf_node):
        #print(leaf_node.x[leaf_node.edge_nodes_x_index][:, :-1])
        #print(target_x)
        nearest_distance = float('inf')
        nearest_node = None
        node = leaf_node
        while(node):
            distance = self._dist_method_2d(
                node.x[node.edge_nodes_x_index][:, :-1], target_x).min()
            if(distance <= nearest_distance):
                nearest_distance = distance
                nearest_node = node
            node = node.parent
        return nearest_node
    def _search_node_has_k(self, nearest_node, k):
        node = nearest_node
        while(node and k > node.x_index.shape[0]):
            #print(node.x_index.shape[0], k)
            node = node.parent
        return node
    def _get_k_index(self, target_x, node, k):
        copied_x = node.x[node.x_index].copy()
        distance_k = self._dist_method_2d(copied_x[:, :-1], target_x)
        res = np.hstack(
                (copied_x[:, -1].reshape(-1, 1), distance_k.reshape(-1, 1)))
        return res[np.argsort(res[:,-1])][0:k, 0]
    def search_nreaest_k(self, target_x, k):
        if(self.dim_len != len(target_x)):
            return None
        nearst_leaf = self._search_nreaest_leaf(target_x, self.root)
        nearest_node = self._search_nreaest_node(target_x, nearst_leaf)
        if(nearest_node.parent):
            #应该使用超圆与超矩形的相交方法，决定是否提升至父亲节点
            nearest_node = nearest_node.parent
        nearest_withk_node = self._search_node_has_k(nearest_node, k)
        if(None == nearest_withk_node):
            print('k too large')
            return None
        return self._get_k_index(target_x, nearest_withk_node, k)
    def show(self):
        print('original x:')
        print(self.x)
        queue = []
        queue.append(self.root)
        while(0 != len(queue)):
            print('edge x')
            print(queue[0].x[queue[0].edge_nodes_x_index])
            print('inner x')
            print(queue[0].x[queue[0].x_index])
            if(queue[0].left):
                queue.append(queue[0].left)
            if(queue[0].right):
                queue.append(queue[0].right)
            del queue[0]

tree = kdTree()
test_x = np.array([
    [2,3],
    [5,4],
    [9,6],
    [4,7],
    [8,1],
    [7,2],
])
tree.fit(test_x)
#tree.show()
tree.search_nreaest_k(np.array([5,7]), 1)



array([3])

* kd树已实现书中所述最近邻搜索，但是无法完成k近邻搜索，按照书中示例，可见是正确的

In [134]:
import numpy as np
from math import *

class KNNClassifier:
    def __init__(self, distance_method = "o"):
        self.kd_tree = None
        if(distance_method == 'm'):
            self._dist_method = self._manhattan_distance
        else:
            self._dist_method = self._euler_dist
    def _euler_dist(self, x1, x):
        return np.sqrt(np.multiply(x-x1, x-x1).sum())
    def _manhattan_distance(self, x1, x):
        return np.abs(x-x1).sum()
    def fit(self, x, y):
        self.x = x
        self.y = y
        self.kd_tree = kdTree()
        self.kd_tree.fit(x)
        return self
    def searchKNeighbour(self, x): 
        pass
test_x = np.array([
    [2,3],
    [5,4],
    [9,6],
    [4,7],
    [8,1],
    [7,2],
])

In [39]:
import numpy as np

class KDTreeNode:
    def __init__(self, x_index_list, x_index,
                attr_index, attr_val, x, parent, isleft):
        '''
        该节点子树的x下标集合
        '''
        self.x_index_list = x_index_list 
        '''
        该节点所持有的分割节点的下标
        '''
        self.x_index = x_index
        '''
        该节点所持有分割节点的分割属性下标
        '''
        self.attr_index = attr_index
        '''
        该节点所持有分割节点的分割属性值
        '''
        self.attr_val = attr_val
        '''
        数据集x的引用
        '''
        self.x = x
        '''
        树结构节点引用
        '''
        self.parent = parent
        self.isleft = isleft
        self.left = None
        self.right = None
class KDTree:
    def __init__(self, x):
        '''
        x: 整型二维列表,ndarray类型
        '''
        if(type(x) != np.ndarray):
            self.x = np.array(x)
        else:
            self.x = x
        self.root = self._build_tree(np.arange(len(self.x)), None, 0, False)
        
    def _search_attr_val(self, x, attr_index):
        return self.median(x[:, attr_index])
    
    def _search_splited_x_index(self, x, attr_index, attr_val):
        attr_list = x[:, attr_index]
        split_x_indexes = np.where(attr_list == attr_val)[0]
        print('in _search_splited_x_index() split_x_indexes ', split_x_indexes)
        split_left_indexes = np.where(attr_list < attr_val)[0]
        print('in _search_splited_x_index() split_left_indexes ', split_left_indexes)
        split_right_indexes = np.where(attr_list > attr_val)[0]
        print('in _search_splited_x_index() split_right_indexes ', split_right_indexes)
        split_right_indexes = np.concatenate((split_right_indexes, split_x_indexes[1:]),
                                            axis = 0)
        return split_x_indexes[0], split_left_indexes, split_right_indexes
    
    def median(self, arr):
        print("in median() sorted arr", sorted(arr))
        return sorted(arr)[int((len(arr) - 1) / 2)]
    
    def _build_tree(self, x_indexes, parent, attr_index, isleft):
        xs = self.x[x_indexes]
        print("in build_tree() attr_index, xs, x_indexes ", attr_index, 
              xs, x_indexes)
        if(len(x_indexes) == 0):
            return None
        attr_val = self._search_attr_val(xs, attr_index)
        print("in build_tree() attr_val ", attr_val)
        x_index, left_indexes, right_indexes = self._search_splited_x_index(
                    xs, attr_index, attr_val)
        x_index = x_indexes[x_index]
        left_indexes = x_indexes[left_indexes]
        right_indexes = x_indexes[right_indexes]
        print("in build_tree() x_index, left_indexes, right_indexes ",
              x_index, left_indexes, right_indexes)
        print("in build_tree() x_indexes, x_index, attr_index, attr_val ",
              x_indexes, x_index, attr_index, attr_val)
        node = KDTreeNode(x_indexes, x_index, attr_index, attr_val, self.x,
                         parent, isleft)
        node.left = self._build_tree(left_indexes, node, (attr_index + 1) % len(self.x[0]), True)
        node.right = self._build_tree(right_indexes, node, (attr_index + 1) % len(self.x[0]), False)
        return node
    def search_nearest_leaf(self, x, root):
        p = root
        attr_index = 0
        while(p):
            if(p.left == None and p.right == None):
                break
            elif(p.left and x[attr_index] < p.attr_val):
                p = p.left
            elif(p.right and x[attr_index] >= p.attr_val):
                p = p.right
            else:
                break
            attr_index = (attr_index + 1) % len(self.x[0])
        print("in search_tree() search %s and result %s result index_x %s" % (
                                            str(x), 
                                            str(self.x[p.x_index]),
                                            str(p.x_index)))
        return p
    
    def dist(self, x1, x2):
        x = x1 - x2
        return np.sqrt(np.sum(x*x))
      
    def search_nearest_node(self, node, goleft, x):
        if(node == None):
            return
        d = self.dist(x, self.x[self.nearest_node.x_index])
        node_d = self.dist(x, self.x[node.x_index])
        if(node_d < d):
            self.nearest_node = node
            d = self.dist(x, self.x[self.nearest_node.x_index])
        attr_d = np.abs(x[node.attr_index] - self.x[node.x_index][node.attr_index])
        print("in search_nearest_node() d %s attr_d %s goleft %s" % (d, attr_d, goleft))
        if(d > attr_d):
            if(goleft):
                temp_nearest_node = self.search_nearest_leaf(x, node.left)
            else:
                temp_nearest_node = self.search_nearest_leaf(x, node.right)
            if(self.dist(x, self.x[temp_nearest_node.x_index]) < d):
                self.nearest_node = temp_nearest_node
        self.search_nearest_node(node.parent, not node.isleft, x)
                
        
    def search_nearest_k(self, x, k): 
        self.nearest_node = self.search_nearest_leaf(x, self.root)
        self.search_nearest_node(self.nearest_node.parent, not self.nearest_node.isleft
                                 , x)
        num = 1
        k_index_list = []
        node_list = [self.nearest_node]
        #默认实现k小的情况，心情不够好，不实现k大的
        if(k < len(self.nearest_node.x_index_list))
            while(num < k and len(node_list) != 0):
                p = node_list.pop(-1)
                
                
            
        print("in search_nearest_k() final nearest_node ", self.x[self.nearest_node.x_index])
        
            
a = [[4,2,3],[1,5,4],[7,5,6],[2,3,1],[6,6,6],[6,6,6]]
b = np.array(a)
b = np.array([
    [2,3],
    [5,4],
    [9,6],
    [4,7],
    [8,1],
    [7,2],
])
tree = KDTree(b)
tree.search_nearest_k([7, 4.5])

in build_tree() attr_index, xs, x_indexes  0 [[2 3]
 [5 4]
 [9 6]
 [4 7]
 [8 1]
 [7 2]] [0 1 2 3 4 5]
in median() sorted arr [2, 4, 5, 7, 8, 9]
in build_tree() attr_val  5
in _search_splited_x_index() split_x_indexes  [1]
in _search_splited_x_index() split_left_indexes  [0 3]
in _search_splited_x_index() split_right_indexes  [2 4 5]
in build_tree() x_index, left_indexes, right_indexes  1 [0 3] [2 4 5]
in build_tree() x_indexes, x_index, attr_index, attr_val  [0 1 2 3 4 5] 1 0 5
in build_tree() attr_index, xs, x_indexes  1 [[2 3]
 [4 7]] [0 3]
in median() sorted arr [3, 7]
in build_tree() attr_val  3
in _search_splited_x_index() split_x_indexes  [0]
in _search_splited_x_index() split_left_indexes  []
in _search_splited_x_index() split_right_indexes  [1]
in build_tree() x_index, left_indexes, right_indexes  0 [] [3]
in build_tree() x_indexes, x_index, attr_index, attr_val  [0 3] 0 1 3
in build_tree() attr_index, xs, x_indexes  0 [] []
in build_tree() attr_index, xs, x_indexes  0 [[4 7]] 

In [90]:
import numpy as np
a=[[4,2,3],[1,5,4],[7,5,6]]
b = np.array(a)
b[[0,1]]
np.median(b[[1,2]][:, 0])


5.5

In [108]:
bb = b[[0,1]][:, 1]
bb
print(len(bb))
sorted(bb)[int(len(bb) / 2 - 1)]


2


2

In [41]:
x1 = np.array([7,4.5])
x2 = np.array([5,4])

x = x1 - x2
np.sqrt(np.sum(x*x))

2.0615528128088303

In [44]:
a = [1,2,3]
a.pop(-1)
a

[1, 2]